In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="meta/llama-3.1-8b-instruct",
  messages=[{"role":"user","content":"Write a limerick about the wonders of GPU computing."}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")



In [ ]:
import torch 


raw_labels = torch.load("/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/logs-api/meta/llama-3.1-8b-instruct/flan_v2/total_output_labels.pt")



In [ ]:
import collections
import matplotlib.pyplot as plt
import torch 

model_name="meta/llama-3.1-8b-instruct"
dataset_name = 'cot'

path = "/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/logs-api/"
raw_labels = torch.load(path + f"{model_name}/{dataset_name}/total_output_labels.pt")

# 统计 labels 列表中的值
label_counts = collections.Counter(raw_labels)

# 打印统计结果
sorted_label_counts = dict(sorted(label_counts.items()))
print("Sorted Label counts:", sorted_label_counts)

# 绘制直方图
labels, counts = zip(*sorted_label_counts.items())

plt.bar(labels, counts, align='center')
plt.xlabel('Labels')
plt.ylabel('Label Counts')
plt.title(f'Labeling Model: {model_name};; Dataset: {dataset_name}; Label Distribution')
plt.xticks(range(min(labels), max(labels) + 1))

plt.text(0.05, 0.95, f'Total Labels: {len(raw_labels)}', horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes)


plt.show()



In [ ]:
from datasets import load_dataset


dataset_name = 'flan_v2'

data = load_dataset('parquet', data_files=f'data_refine/tulu_split_parquet/{dataset_name}.parquet', split=None, cache_dir=None)

dialogs = data['train']

In [ ]:
data

In [ ]:
import os
import torch

# 文件夹路径
folder_path = "/home/azureuser/cloudfiles/code/Users/jinlong.pang/LADR_LLM_alignment_data_refinement/open-instruct/logs-api/meta/llama-3.1-8b-instruct/cot"
incorrect_files = []

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    if filename.startswith("output_labels_") and filename.endswith(".pt"):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # 加载文件内容
            labels = torch.load(file_path)
            
            # 检查文件长度是否等于 1024
            if len(labels) != 1024:
                incorrect_files.append(filename)
                print(f"{filename}: {len(labels)} labels (incorrect length)")
            # else:
                # print(f"{filename}: {len(labels)} labels (correct length)")
        
        except Exception as e:
            print(f"Error loading {filename}: {e}")


In [ ]:
from huggingface_hub import HfApi, upload_file, upload_folder

# 第一步：创建 Hugging Face API 实例
api = HfApi()

# 第二步：创建数据集仓库
# 将 'username' 替换为你的 Hugging Face 用户名，'dataset_name' 替换为你想要创建的数据集名称
repo_id = "jlpang888/LLM_Data_selection"  
# api.create_repo(repo_id=repo_id, repo_type="dataset")

# 第三步：上传文件
# 指定本地文件路径和上传路径

file_path = "model_finetune_cluster/new_train_data/mistralai/Mistral-7B-Instruct-v0.3/all_train/filtered-cured_dataset.json"  # 本地数据文件路径
path_in_repo = "mistral_labeling_filtered-cured.json"            # 上传到 Hugging Face 数据集中显示的路径和文件名
api.upload_file(
    path_or_fileobj=file_path,
    path_in_repo=path_in_repo,
    repo_id=repo_id,
    repo_type="dataset"
)

# 如果你想上传一个文件夹而不是单个文件，可以使用 upload_folder 函数：
# folder_path = "model_finetune_cluster/new_train_data"  # 你的文件夹路径
# upload_folder(
#     folder_path=folder_path,
#     repo_id=repo_id,  # 上传到相同的 repo_id
#     repo_type="dataset"
# )

print(f"数据集已成功上传到 Hugging Face: https://huggingface.co/datasets/{repo_id}")


In [4]:
from huggingface_hub import HfApi, upload_folder
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm
import os

# 创建 Hugging Face Hub 的 API 实例
api = HfApi()

# 创建模型仓库（如果仓库已经存在，可以跳过这一步）
repo_name = "jlpang888/70B-finetune-filtered"  # 替换为你的用户名和模型名称
api.create_repo(repo_id=repo_name, exist_ok=True)  # exist_ok=True 表示仓库已存在时不会报错

# 模型文件夹路径
model_path = "finetune_70B_result/models/meta-llama/Meta-Llama-3.1-8B-Instruct/all_train/meta-llama/Meta-Llama-3-70B/"

# 计算文件总数，用于进度条
total_files = sum([len(files) for _, _, files in os.walk(model_path)])

# 自定义 upload_folder 方法，带有进度条
def upload_with_progress(folder_path, repo_id):
    # 创建一个 tqdm 进度条
    with tqdm(total=total_files, desc="Uploading", unit="file") as pbar:
        # 遍历每个文件并上传
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # 上传每个文件
                api.upload_file(
                    path_or_fileobj=file_path,
                    path_in_repo=file_path.replace(folder_path, ""),  # 去掉本地路径前缀
                    repo_id=repo_id
                )
                # 每上传一个文件，进度条更新
                pbar.update(1)

# 调用带有进度条的上传函数
upload_with_progress(model_path, repo_name)


In [1]:
from azure.storage.blob import BlobServiceClient
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import HfApi, HfFolder
import os

# 初始化 Azure Blob Storage 客户端
connection_string = "DefaultEndpointsProtocol=https;AccountName=afminternshipuksouth;AccountKey=BaMauNxU5/sAzzv5Cd3KWHR1qr8YVe3i0LeFEbSc0h3BC3ssvJMbCvLRrWqX9UVIU70z9bVdedh4+AStVF2kEw==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_name = "jinlong"
blob_prefix = "finetune_70B_result/models/meta-llama/Meta-Llama-3.1-8B-Instruct/all_train/meta-llama/Meta-Llama-3-70B/lora_merged_full"

# Hugging Face Hub 配置
api = HfApi()
repo_name = "jlpang888/70B-finetune-full"
api.create_repo(repo_id=repo_name, exist_ok=True)

# 获取 Blob 容器
container_client = blob_service_client.get_container_client(container_name)

# 模型文件的流式处理逻辑
blobs_list = container_client.list_blobs(name_starts_with=blob_prefix)

for blob in blobs_list:
    blob_client = container_client.get_blob_client(blob)
    
    # 在这里流式读取 blob 数据
    stream = blob_client.download_blob()

    # 将数据直接上传到 Hugging Face (理论上分片上传)
    # 使用 stream.read() 逐块处理数据并上传
    # 你可以将这些流数据直接传递给 `push_to_hub` 的某个子方法（目前 Hugging Face 不支持这个直接接口，需要定制逻辑）

    # 此处是伪代码，表示你可以逐块上传文件内容，而非完整下载到本地再上传
    for chunk in stream.chunks():
        # 你可以在这里处理分块上传，避免完整下载到本地
        pass


/home/azureuser/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 